# Imports

In [1]:
import pandas as pd
import seaborn as sns
import sklearn as sk
import datetime as dt
import sys
import os
import twitch
import json

# To ignore annoying warning
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
warnings.filterwarnings("ignore", category=DeprecationWarning)
print('Deprecation warning will be ignored!')

Deprecation warning will be ignored!


# Account info 

In [2]:
# Username of your channel
username = 'Test_Bot_AI'
# oauth to get on twitch.com
password = 'oauth:xxdf0t7dc191uc8bv3tb8szudxlnbm'

# Turning Sardo lovely chat in a dataframe

In [3]:
data_sardo = pd.read_csv('/Users/theophanegregoir/Desktop/TwitchFun/twitch-ai/data/csv/sardo_live_nocturne.csv')
data_sardo = data_sardo.drop(columns=['Unnamed: 0'])
data_sardo = data_sardo.dropna()
data_sardo

,Time,Sender,Sender_type,Content
0,0:00:05,SyIvanus,subscriber,Ohhhhhhhhhhh oauis
1,0:00:06,Bleubz,subscriber,OUI
2,0:00:08,Reykin,subscriber,sardYop
3,0:00:19,Crapps007,subscriber,y'a pas de son :(
4,0:00:26,leboeufbourguignons,subscriber,OUUUIIIIIIIIIIIIIIIIII sardDodo sardDodo sardD...
...,...,...,...,...
8356,2:48:43,eldebor_zat,none,rip
8357,2:48:43,Majin_Uub00,subscriber,les bielorusse encore
8358,2:48:43,gazoline42,none,mdr
8359,2:48:44,real_pandaman,subscriber,coup dur pour le joueur fr


# Dealing with emojis

In [4]:
from typing import List

In [5]:
def _get_only_content(df: pd.DataFrame) -> List:
    df_temp = df[['Content']]
    df_temp = df_temp.dropna()
    return list(df_temp.values)

content_list = _get_only_content(data_sardo)


In [6]:
emoji2unicode_name = {
    '😂': 'FACE WITH TEARS OF JOY',
    '❤': 'HEAVY BLACK HEART',
    '♥': 'BLACK HEART SUIT',
    '😍': 'SMILING FACE WITH HEART-SHAPED EYES',
    '😭': 'LOUDLY CRYING FACE',
    '😘': 'FACE THROWING A KISS',
    '😊': 'SMILING FACE WITH SMILING EYES',
    '👌': 'OK HAND SIGN',
    '💕': 'TWO HEARTS',
    '👏': 'CLAPPING HANDS SIGN',
    '😁': 'GRINNING FACE WITH SMILING EYES',
    '☺': 'WHITE SMILING FACE',
    '♡': 'WHITE HEART SUIT',
    '👍': 'THUMBS UP SIGN',
    '😩': 'WEARY FACE',
    '🙏': 'PERSON WITH FOLDED HANDS',
    '✌': 'VICTORY HAND',
    '😏': 'SMIRKING FACE',
    '😉': 'WINKING FACE',
    '🙌': 'PERSON RAISING BOTH HANDS IN CELEBRATION',
    '🙈': 'SEE-NO-EVIL MONKEY',
    '💪': 'FLEXED BICEPS',
    '😄': 'SMILING FACE WITH OPEN MOUTH AND SMILING EYES',
    '😒': 'UNAMUSED FACE',
    '💃': 'DANCER',
    '💖': 'SPARKLING HEART',
    '😃': 'SMILING FACE WITH OPEN MOUTH',
    '😔': 'PENSIVE FACE',
    '😱': 'FACE SCREAMING IN FEAR',
    '🎉': 'PARTY POPPER',
    '😜': 'FACE WITH STUCK-OUT TONGUE AND WINKING EYE',
    '☯': 'YIN YANG',
    '🌸': 'CHERRY BLOSSOM',
    '💜': 'PURPLE HEART',
    '💙': 'BLUE HEART',
    '✨': 'SPARKLES',
    '😳': 'FLUSHED FACE',
    '💗': 'GROWING HEART',
    '★': 'BLACK STAR',
    '█': 'FULL BLOCK',
    '☀': 'BLACK SUN WITH RAYS',
    '😡': 'POUTING FACE',
    '😎': 'SMILING FACE WITH SUNGLASSES',
    '😢': 'CRYING FACE',
    '💋': 'KISS MARK',
    '😋': 'FACE SAVOURING DELICIOUS FOOD',
    '🙊': 'SPEAK-NO-EVIL MONKEY',
    '😴': 'SLEEPING FACE',
    '🎶': 'MULTIPLE MUSICAL NOTES',
    '💞': 'REVOLVING HEARTS',
    '😌': 'RELIEVED FACE',
    '🔥': 'FIRE',
    '💯': 'HUNDRED POINTS SYMBOL',
    '🔫': 'PISTOL',
    '💛': 'YELLOW HEART',
    '💁': 'INFORMATION DESK PERSON',
    '💚': 'GREEN HEART',
    '♫': 'BEAMED EIGHTH NOTES',
    '😞': 'DISAPPOINTED FACE',
    '😆': 'SMILING FACE WITH OPEN MOUTH AND TIGHTLY-CLOSED EYES',
    '😝': 'FACE WITH STUCK-OUT TONGUE AND TIGHTLY-CLOSED EYES',
    '😪': 'SLEEPY FACE',
    '�': 'REPLACEMENT CHARACTER',
    '😫': 'TIRED FACE',
    '😅': 'SMILING FACE WITH OPEN MOUTH AND COLD SWEAT',
    '👊': 'FISTED HAND SIGN',
    '💀': 'SKULL',
    '😀': 'GRINNING FACE',
    '😚': 'KISSING FACE WITH CLOSED EYES',
    '😻': 'SMILING CAT FACE WITH HEART-SHAPED EYES',
    '©': 'COPYRIGHT SIGN',
    '👀': 'EYES',
    '💘': 'HEART WITH ARROW',
    '🐓': 'ROOSTER',
    '☕': 'HOT BEVERAGE',
    '👋': 'WAVING HAND SIGN',
    '✋': 'RAISED HAND',
    '🎊': 'CONFETTI BALL',
    '🍕': 'SLICE OF PIZZA',
    '❄': 'SNOWFLAKE',
    '😥': 'DISAPPOINTED BUT RELIEVED FACE',
    '😕': 'CONFUSED FACE',
    '💥': 'COLLISION SYMBOL',
    '💔': 'BROKEN HEART',
    '😤': 'FACE WITH LOOK OF TRIUMPH',
    '😈': 'SMILING FACE WITH HORNS',
    '►': 'BLACK RIGHT-POINTING POINTER',
    '✈': 'AIRPLANE',
    '🔝': 'TOP WITH UPWARDS ARROW ABOVE',
    '😰': 'FACE WITH OPEN MOUTH AND COLD SWEAT',
    '⚽': 'SOCCER BALL',
    '😑': 'EXPRESSIONLESS FACE',
    '👑': 'CROWN',
    '😹': 'CAT FACE WITH TEARS OF JOY',
    '👉': 'WHITE RIGHT POINTING BACKHAND INDEX',
    '🍃': 'LEAF FLUTTERING IN WIND',
    '🎁': 'WRAPPED PRESENT',
    '😠': 'ANGRY FACE',
    '🐧': 'PENGUIN',
    '☆': 'WHITE STAR',
    '🍀': 'FOUR LEAF CLOVER',
    '🎈': 'BALLOON',
    '🎅': 'FATHER CHRISTMAS',
    '😓': 'FACE WITH COLD SWEAT',
    '😣': 'PERSEVERING FACE',
    '😐': 'NEUTRAL FACE',
    '✊': 'RAISED FIST',
    '😨': 'FEARFUL FACE',
    '😖': 'CONFOUNDED FACE',
    '💤': 'SLEEPING SYMBOL',
    '💓': 'BEATING HEART',
    '👎': 'THUMBS DOWN SIGN',
    '💦': 'SPLASHING SWEAT SYMBOL',
    '✔': 'HEAVY CHECK MARK',
    '😷': 'FACE WITH MEDICAL MASK',
    '⚡': 'HIGH VOLTAGE SIGN',
    '🙋': 'HAPPY PERSON RAISING ONE HAND',
    '🎄': 'CHRISTMAS TREE',
    '💩': 'PILE OF POO',
    '🎵': 'MUSICAL NOTE',
    '➡': 'BLACK RIGHTWARDS ARROW',
    '😛': 'FACE WITH STUCK-OUT TONGUE',
    '😬': 'GRIMACING FACE',
    '👯': 'WOMAN WITH BUNNY EARS',
    '💎': 'GEM STONE',
    '🌿': 'HERB',
    '🎂': 'BIRTHDAY CAKE',
    '🌟': 'GLOWING STAR',
    '🔮': 'CRYSTAL BALL',
    '❗': 'HEAVY EXCLAMATION MARK SYMBOL',
    '👫': 'MAN AND WOMAN HOLDING HANDS',
    '🏆': 'TROPHY',
    '✖': 'HEAVY MULTIPLICATION X',
    '☝': 'WHITE UP POINTING INDEX',
    '😙': 'KISSING FACE WITH SMILING EYES',
    '⛄': 'SNOWMAN WITHOUT SNOW',
    '👅': 'TONGUE',
    '♪': 'EIGHTH NOTE',
    '🍂': 'FALLEN LEAF',
    '💏': 'KISS',
    '🔪': 'HOCHO',
    '🌴': 'PALM TREE',
    '👈': 'WHITE LEFT POINTING BACKHAND INDEX',
    '🌹': 'ROSE',
    '🙆': 'FACE WITH OK GESTURE',
    '➜': 'HEAVY ROUND-TIPPED RIGHTWARDS ARROW',
    '👻': 'GHOST',
    '💰': 'MONEY BAG',
    '🍻': 'CLINKING BEER MUGS',
    '🙅': 'FACE WITH NO GOOD GESTURE',
    '🌞': 'SUN WITH FACE',
    '🍁': 'MAPLE LEAF',
    '⭐': 'WHITE MEDIUM STAR',
    '▪': 'BLACK SMALL SQUARE',
    '🎀': 'RIBBON',
    '━': 'BOX DRAWINGS HEAVY HORIZONTAL',
    '☷': 'TRIGRAM FOR EARTH',
    '🐷': 'PIG FACE',
    '🙉': 'HEAR-NO-EVIL MONKEY',
    '🌺': 'HIBISCUS',
    '💅': 'NAIL POLISH',
    '🐶': 'DOG FACE',
    '🌚': 'NEW MOON WITH FACE',
    '👽': 'EXTRATERRESTRIAL ALIEN',
    '🎤': 'MICROPHONE',
    '👭': 'TWO WOMEN HOLDING HANDS',
    '🎧': 'HEADPHONE',
    '👆': 'WHITE UP POINTING BACKHAND INDEX',
    '🍸': 'COCKTAIL GLASS',
    '🍷': 'WINE GLASS',
    '®': 'REGISTERED SIGN',
    '🍉': 'WATERMELON',
    '😇': 'SMILING FACE WITH HALO',
    '☑': 'BALLOT BOX WITH CHECK',
    '🏃': 'RUNNER',
    '😿': 'CRYING CAT FACE',
    '│': 'BOX DRAWINGS LIGHT VERTICAL',
    '💣': 'BOMB',
    '🍺': 'BEER MUG',
    '▶': 'BLACK RIGHT-POINTING TRIANGLE',
    '😲': 'ASTONISHED FACE',
    '🎸': 'GUITAR',
    '🍹': 'TROPICAL DRINK',
    '💫': 'DIZZY SYMBOL',
    '📚': 'BOOKS',
    '😶': 'FACE WITHOUT MOUTH',
    '🌷': 'TULIP',
    '💝': 'HEART WITH RIBBON',
    '💨': 'DASH SYMBOL',
    '🏈': 'AMERICAN FOOTBALL',
    '💍': 'RING',
    '☔': 'UMBRELLA WITH RAIN DROPS',
    '👸': 'PRINCESS',
    '🇪': 'REGIONAL INDICATOR SYMBOL LETTER E',
    '░': 'LIGHT SHADE',
    '🍩': 'DOUGHNUT',
    '👾': 'ALIEN MONSTER',
    '☁': 'CLOUD',
    '🌻': 'SUNFLOWER',
    '😵': 'DIZZY FACE',
    '📒': 'LEDGER',
    '↿': 'UPWARDS HARPOON WITH BARB LEFTWARDS',
    '🐯': 'TIGER FACE',
    '👼': 'BABY ANGEL',
    '🍔': 'HAMBURGER',
    '😸': 'GRINNING CAT FACE WITH SMILING EYES',
    '👶': 'BABY',
    '↾': 'UPWARDS HARPOON WITH BARB RIGHTWARDS',
    '💐': 'BOUQUET',
    '🌊': 'WATER WAVE',
    '🍦': 'SOFT ICE CREAM',
    '🍓': 'STRAWBERRY',
    '👇': 'WHITE DOWN POINTING BACKHAND INDEX',
    '💆': 'FACE MASSAGE',
    '🍴': 'FORK AND KNIFE',
    '😧': 'ANGUISHED FACE',
    '🇸': 'REGIONAL INDICATOR SYMBOL LETTER S',
    '😮': 'FACE WITH OPEN MOUTH',
    '▓': 'DARK SHADE',
    '🚫': 'NO ENTRY SIGN',
    '😽': 'KISSING CAT FACE WITH CLOSED EYES',
    '🌈': 'RAINBOW',
    '🙀': 'WEARY CAT FACE',
    '⚠': 'WARNING SIGN',
    '🎮': 'VIDEO GAME',
    '╯': 'BOX DRAWINGS LIGHT ARC UP AND LEFT',
    '🍆': 'AUBERGINE',
    '🍰': 'SHORTCAKE',
    '✓': 'CHECK MARK',
    '👐': 'OPEN HANDS SIGN',
    '🙇': 'PERSON BOWING DEEPLY',
    '🍟': 'FRENCH FRIES',
    '🍌': 'BANANA',
    '💑': 'COUPLE WITH HEART',
    '👬': 'TWO MEN HOLDING HANDS',
    '🐣': 'HATCHING CHICK',
    '🎃': 'JACK-O-LANTERN',
    '▬': 'BLACK RECTANGLE',
    '￼': 'OBJECT REPLACEMENT CHARACTER',
    '😟': 'WORRIED FACE',
    '🐾': 'PAW PRINTS',
    '🎓': 'GRADUATION CAP',
    '🏊': 'SWIMMER',
    '🍫': 'CHOCOLATE BAR',
    '📷': 'CAMERA',
    '👄': 'MOUTH',
    '🌼': 'BLOSSOM',
    '🚶': 'PEDESTRIAN',
    '🐱': 'CAT FACE',
    '║': 'BOX DRAWINGS DOUBLE VERTICAL',
    '🐸': 'FROG FACE',
    '🇺': 'REGIONAL INDICATOR SYMBOL LETTER U',
    '👿': 'IMP',
    '🚬': 'SMOKING SYMBOL',
    '✿': 'BLACK FLORETTE',
    '📖': 'OPEN BOOK',
    '🐒': 'MONKEY',
    '🌍': 'EARTH GLOBE EUROPE-AFRICA',
    '┊': 'BOX DRAWINGS LIGHT QUADRUPLE DASH VERTICAL',
    '🐥': 'FRONT-FACING BABY CHICK',
    '🌀': 'CYCLONE',
    '🐼': 'PANDA FACE',
    '🎥': 'MOVIE CAMERA',
    '💄': 'LIPSTICK',
    '💸': 'MONEY WITH WINGS',
    '⛔': 'NO ENTRY',
    '●': 'BLACK CIRCLE',
    '🏀': 'BASKETBALL AND HOOP',
    '💉': 'SYRINGE',
    '💟': 'HEART DECORATION',
    '🚗': 'AUTOMOBILE',
    '😯': 'HUSHED FACE',
    '📝': 'MEMO',
    '═': 'BOX DRAWINGS DOUBLE HORIZONTAL',
    '♦': 'BLACK DIAMOND SUIT',
    '💭': 'THOUGHT BALLOON',
    '🌙': 'CRESCENT MOON',
    '🐟': 'FISH',
    '👣': 'FOOTPRINTS',
    '☞': 'WHITE RIGHT POINTING INDEX',
    '✂': 'BLACK SCISSORS',
    '🗿': 'MOYAI',
    '🍝': 'SPAGHETTI',
    '👪': 'FAMILY',
    '🍭': 'LOLLIPOP',
    '🌃': 'NIGHT WITH STARS',
    '❌': 'CROSS MARK',
    '🐰': 'RABBIT FACE',
    '💊': 'PILL',
    '🚨': 'POLICE CARS REVOLVING LIGHT',
    '😦': 'FROWNING FACE WITH OPEN MOUTH',
    '🍪': 'COOKIE',
    '🍣': 'SUSHI',
    '╭': 'BOX DRAWINGS LIGHT ARC DOWN AND RIGHT',
    '✧': 'WHITE FOUR POINTED STAR',
    '🎆': 'FIREWORKS',
    '╮': 'BOX DRAWINGS LIGHT ARC DOWN AND LEFT',
    '🎎': 'JAPANESE DOLLS',
    '🇩': 'REGIONAL INDICATOR SYMBOL LETTER D',
    '✅': 'WHITE HEAVY CHECK MARK',
    '👹': 'JAPANESE OGRE',
    '📱': 'MOBILE PHONE',
    '🙍': 'PERSON FROWNING',
    '🍑': 'PEACH',
    '🎼': 'MUSICAL SCORE',
    '🔊': 'SPEAKER WITH THREE SOUND WAVES',
    '🌌': 'MILKY WAY',
    '🍎': 'RED APPLE',
    '🐻': 'BEAR FACE',
    '─': 'BOX DRAWINGS LIGHT HORIZONTAL',
    '╰': 'BOX DRAWINGS LIGHT ARC UP AND RIGHT',
    '💇': 'HAIRCUT',
    '♬': 'BEAMED SIXTEENTH NOTES',
    '♚': 'BLACK CHESS KING',
    '🔴': 'LARGE RED CIRCLE',
    '🍱': 'BENTO BOX',
    '🍊': 'TANGERINE',
    '🍒': 'CHERRIES',
    '🐭': 'MOUSE FACE',
    '👟': 'ATHLETIC SHOE',
    '🌎': 'EARTH GLOBE AMERICAS',
    '🍍': 'PINEAPPLE',
    '🐮': 'COW FACE',
    '📲': 'MOBILE PHONE WITH RIGHTWARDS ARROW AT LEFT',
    '☼': 'WHITE SUN WITH RAYS',
    '🌅': 'SUNRISE',
    '🇷': 'REGIONAL INDICATOR SYMBOL LETTER R',
    '👠': 'HIGH-HEELED SHOE',
    '🌽': 'EAR OF MAIZE',
    '💧': 'DROPLET',
    '❓': 'BLACK QUESTION MARK ORNAMENT',
    '🍬': 'CANDY',
    '😺': 'SMILING CAT FACE WITH OPEN MOUTH',
    '🐴': 'HORSE FACE',
    '🚀': 'ROCKET',
    '¦': 'BROKEN BAR',
    '💢': 'ANGER SYMBOL',
    '🎬': 'CLAPPER BOARD',
    '🍧': 'SHAVED ICE',
    '🍜': 'STEAMING BOWL',
    '🐏': 'RAM',
    '🐘': 'ELEPHANT',
    '👧': 'GIRL',
    '⠀': 'BRAILLE PATTERN BLANK',
    '🏄': 'SURFER',
    '➤': 'BLACK RIGHTWARDS ARROWHEAD',
    '⬆': 'UPWARDS BLACK ARROW',
    '🍋': 'LEMON',
    '🆗': 'SQUARED OK',
    '⚪': 'MEDIUM WHITE CIRCLE',
    '📺': 'TELEVISION',
    '🍅': 'TOMATO',
    '⛅': 'SUN BEHIND CLOUD',
    '🐢': 'TURTLE',
    '👙': 'BIKINI',
    '🏡': 'HOUSE WITH GARDEN',
    '🌾': 'EAR OF RICE',
    '◉': 'FISHEYE',
    '✏': 'PENCIL',
    '🐬': 'DOLPHIN',
    '🍤': 'FRIED SHRIMP',
    '🇹': 'REGIONAL INDICATOR SYMBOL LETTER T',
    '♣': 'BLACK CLUB SUIT',
    '🐝': 'HONEYBEE',
    '🌝': 'FULL MOON WITH FACE',
    '🇮': 'REGIONAL INDICATOR SYMBOL LETTER I',
    '🔋': 'BATTERY',
    '🐍': 'SNAKE',
    '♔': 'WHITE CHESS KING',
    '🍳': 'COOKING',
    '🔵': 'LARGE BLUE CIRCLE',
    '😾': 'POUTING CAT FACE',
    '🌕': 'FULL MOON SYMBOL',
    '🐨': 'KOALA',
    '🔐': 'CLOSED LOCK WITH KEY',
    '💿': 'OPTICAL DISC',
    '❁': 'EIGHT PETALLED OUTLINED BLACK FLORETTE',
    '🌳': 'DECIDUOUS TREE',
    '👰': 'BRIDE WITH VEIL',
    '❀': 'WHITE FLORETTE',
    '⚓': 'ANCHOR',
    '🚴': 'BICYCLIST',
    '▀': 'UPPER HALF BLOCK',
    '👗': 'DRESS',
    '➕': 'HEAVY PLUS SIGN',
    '💬': 'SPEECH BALLOON',
    '▒': 'MEDIUM SHADE',
    '🔜': 'SOON WITH RIGHTWARDS ARROW ABOVE',
    '🍨': 'ICE CREAM',
    '💲': 'HEAVY DOLLAR SIGN',
    '⛽': 'FUEL PUMP',
    '🍙': 'RICE BALL',
    '🍗': 'POULTRY LEG',
    '🍲': 'POT OF FOOD',
    '🍥': 'FISH CAKE WITH SWIRL DESIGN',
    '▸': 'BLACK RIGHT-POINTING SMALL TRIANGLE',
    '♛': 'BLACK CHESS QUEEN',
    '😼': 'CAT FACE WITH WRY SMILE',
    '🐙': 'OCTOPUS',
    '👨': 'MAN',
    '🍚': 'COOKED RICE',
    '🍖': 'MEAT ON BONE',
    '♨': 'HOT SPRINGS',
    '🎹': 'MUSICAL KEYBOARD',
    '♕': 'WHITE CHESS QUEEN',
    '▃': 'LOWER THREE EIGHTHS BLOCK',
    '🚘': 'ONCOMING AUTOMOBILE',
    '🍏': 'GREEN APPLE',
    '👩': 'WOMAN',
    '👦': 'BOY',
    '🇬': 'REGIONAL INDICATOR SYMBOL LETTER G',
    '🇧': 'REGIONAL INDICATOR SYMBOL LETTER B',
    '☠': 'SKULL AND CROSSBONES',
    '🐠': 'TROPICAL FISH',
    '🚹': 'MENS SYMBOL',
    '💵': 'BANKNOTE WITH DOLLAR SIGN',
    '✰': 'SHADOWED WHITE STAR',
    '╠': 'BOX DRAWINGS DOUBLE VERTICAL AND RIGHT',
    '👛': 'PURSE',
    '🚙': 'RECREATIONAL VEHICLE',
    '🌱': 'SEEDLING',
    '💻': 'PERSONAL COMPUTER',
    '🌏': 'EARTH GLOBE ASIA-AUSTRALIA',
    '▄': 'LOWER HALF BLOCK',
    '👓': 'EYEGLASSES',
    '◄': 'BLACK LEFT-POINTING POINTER',
    '⚾': 'BASEBALL',
    '🌲': 'EVERGREEN TREE',
    '👴': 'OLDER MAN',
    '🏠': 'HOUSE BUILDING',
    '🍇': 'GRAPES',
    '🍘': 'RICE CRACKER',
    '🍛': 'CURRY AND RICE',
    '🐇': 'RABBIT',
    '🔞': 'NO ONE UNDER EIGHTEEN SYMBOL',
    '👵': 'OLDER WOMAN',
    '◀': 'BLACK LEFT-POINTING TRIANGLE',
    '🔙': 'BACK WITH LEFTWARDS ARROW ABOVE',
    '🌵': 'CACTUS',
    '🐽': 'PIG NOSE',
    '🍮': 'CUSTARD',
    '🎇': 'FIREWORK SPARKLER',
    '🐎': 'HORSE',
    '➔': 'HEAVY WIDE-HEADED RIGHTWARDS ARROW',
    '💶': 'BANKNOTE WITH EURO SIGN',
    '🐤': 'BABY CHICK',
    '╩': 'BOX DRAWINGS DOUBLE UP AND HORIZONTAL',
    '🛀': 'BATH',
    '🌑': 'NEW MOON SYMBOL',
    '🚲': 'BICYCLE',
    '🐑': 'SHEEP',
    '🏁': 'CHEQUERED FLAG',
    '🍞': 'BREAD',
    '🎾': 'TENNIS RACQUET AND BALL',
    '╚': 'BOX DRAWINGS DOUBLE UP AND RIGHT',
    '🈹': 'SQUARED CJK UNIFIED IDEOGRAPH-5272',
    '🐳': 'SPOUTING WHALE',
    '👮': 'POLICE OFFICER',
    '☹': 'WHITE FROWNING FACE',
    '🐵': 'MONKEY FACE',
    '✪': 'CIRCLED WHITE STAR',
    '◕': 'CIRCLE WITH ALL BUT UPPER LEFT QUADRANT BLACK',
    '🗼': 'TOKYO TOWER',
    '▐': 'RIGHT HALF BLOCK',
    '♠': 'BLACK SPADE SUIT',
    '┳': 'BOX DRAWINGS HEAVY DOWN AND HORIZONTAL',
    '👺': 'JAPANESE GOBLIN',
    '🐚': 'SPIRAL SHELL',
    '👂': 'EAR',
    '🗽': 'STATUE OF LIBERTY',
    '🍵': 'TEACUP WITHOUT HANDLE',
    '🆒': 'SQUARED COOL',
    '🍯': 'HONEY POT',
    '🐺': 'WOLF FACE',
    '⇨': 'RIGHTWARDS WHITE ARROW',
    '➨': 'HEAVY CONCAVE-POINTED BLACK RIGHTWARDS ARROW',
    '🌓': 'FIRST QUARTER MOON SYMBOL',
    '🔒': 'LOCK',
    '╬': 'BOX DRAWINGS DOUBLE VERTICAL AND HORIZONTAL',
    '👳': 'MAN WITH TURBAN',
    '🌂': 'CLOSED UMBRELLA',
    '🚌': 'BUS',
    '♩': 'QUARTER NOTE',
    '🍡': 'DANGO',
    '❥': 'ROTATED HEAVY BLACK HEART BULLET',
    '🎡': 'FERRIS WHEEL',
    '💌': 'LOVE LETTER',
    '🐩': 'POODLE',
    '🌜': 'LAST QUARTER MOON WITH FACE',
    '⌚': 'WATCH',
    '🚿': 'SHOWER',
    '🐖': 'PIG',
    '🔆': 'HIGH BRIGHTNESS SYMBOL',
    '🌛': 'FIRST QUARTER MOON WITH FACE',
    '💂': 'GUARDSMAN',
    '🐔': 'CHICKEN',
    '🙎': 'PERSON WITH POUTING FACE',
    '🏩': 'LOVE HOTEL',
    '🇫': 'REGIONAL INDICATOR SYMBOL LETTER F',
    '🔨': 'HAMMER',
    '📢': 'PUBLIC ADDRESS LOUDSPEAKER',
    '🐦': 'BIRD',
    '🐲': 'DRAGON FACE',
    '♻': 'BLACK UNIVERSAL RECYCLING SYMBOL',
    '🌘': 'WANING CRESCENT MOON SYMBOL',
    '🍐': 'PEAR',
    '🌔': 'WAXING GIBBOUS MOON SYMBOL',
    '╥': 'BOX DRAWINGS DOWN DOUBLE AND HORIZONTAL SINGLE',
    '❊': 'EIGHT TEARDROP-SPOKED PROPELLER ASTERISK',
    '👖': 'JEANS',
    '🚺': 'WOMENS SYMBOL',
    '😗': 'KISSING FACE',
    '🎭': 'PERFORMING ARTS',
    '🐄': 'COW',
    '◟': 'LOWER LEFT QUADRANT CIRCULAR ARC',
    '🍢': 'ODEN',
    '🎨': 'ARTIST PALETTE',
    '⬇': 'DOWNWARDS BLACK ARROW',
    '🚼': 'BABY SYMBOL',
    '⛲': 'FOUNTAIN',
    '▁': 'LOWER ONE EIGHTH BLOCK',
    '🇴': 'REGIONAL INDICATOR SYMBOL LETTER O',
    '🌗': 'LAST QUARTER MOON SYMBOL',
    '🌖': 'WANING GIBBOUS MOON SYMBOL',
    '🔅': 'LOW BRIGHTNESS SYMBOL',
    '👜': 'HANDBAG',
    '🐌': 'SNAIL',
    '💼': 'BRIEFCASE',
    '🚕': 'TAXI',
    '🐹': 'HAMSTER FACE',
    '🌠': 'SHOOTING STAR',
    '🐈': 'CAT',
    '⇧': 'UPWARDS WHITE ARROW',
    '☎': 'BLACK TELEPHONE',
    '🌁': 'FOGGY',
    '⚫': 'MEDIUM BLACK CIRCLE',
    '♧': 'WHITE CLUB SUIT',
    '🏰': 'EUROPEAN CASTLE',
    '🚵': 'MOUNTAIN BICYCLIST',
    '🎢': 'ROLLER COASTER',
    '🎷': 'SAXOPHONE',
    '🎐': 'WIND CHIME',
    '┈': 'BOX DRAWINGS LIGHT QUADRUPLE DASH HORIZONTAL',
    '╗': 'BOX DRAWINGS DOUBLE DOWN AND LEFT',
    '╱': 'BOX DRAWINGS LIGHT DIAGONAL UPPER RIGHT TO LOWER LEFT',
    '🌇': 'SUNSET OVER BUILDINGS',
    '⏰': 'ALARM CLOCK',
    '⇩': 'DOWNWARDS WHITE ARROW',
    '🚂': 'STEAM LOCOMOTIVE',
    '◠': 'UPPER HALF CIRCLE',
    '🎿': 'SKI AND SKI BOOT',
    '✦': 'BLACK FOUR POINTED STAR',
    '🆔': 'SQUARED ID',
    '⛪': 'CHURCH',
    '🌒': 'WAXING CRESCENT MOON SYMBOL',
    '🐪': 'DROMEDARY CAMEL',
    '╔': 'BOX DRAWINGS DOUBLE DOWN AND RIGHT',
    '╝': 'BOX DRAWINGS DOUBLE UP AND LEFT',
    '👔': 'NECKTIE',
    '🔱': 'TRIDENT EMBLEM',
    '🆓': 'SQUARED FREE',
    '🐋': 'WHALE',
    '▽': 'WHITE DOWN-POINTING TRIANGLE',
    '▂': 'LOWER ONE QUARTER BLOCK',
    '🐛': 'BUG',
    '👕': 'T-SHIRT',
    '🚋': 'TRAM CAR',
    '💳': 'CREDIT CARD',
    '🌆': 'CITYSCAPE AT DUSK',
    '🏧': 'AUTOMATED TELLER MACHINE',
    '💡': 'ELECTRIC LIGHT BULB',
    '🔹': 'SMALL BLUE DIAMOND',
    '⬅': 'LEFTWARDS BLACK ARROW',
    '🍠': 'ROASTED SWEET POTATO',
    '🐫': 'BACTRIAN CAMEL',
    '🏪': 'CONVENIENCE STORE',
    '۩': 'ARABIC PLACE OF SAJDAH',
    '🇱': 'REGIONAL INDICATOR SYMBOL LETTER L',
    '📹': 'VIDEO CAMERA',
    '👞': 'MANS SHOE',
    '🚑': 'AMBULANCE',
    '🆘': 'SQUARED SOS',
    '👚': 'WOMANS CLOTHES',
    '🚍': 'ONCOMING BUS',
    '□': 'WHITE SQUARE',
    '🐂': 'OX',
    '🚣': 'ROWBOAT',
    '✳': 'EIGHT SPOKED ASTERISK',
    '🏉': 'RUGBY FOOTBALL',
    '🗻': 'MOUNT FUJI',
    '🐀': 'RAT',
    '╦': 'BOX DRAWINGS DOUBLE DOWN AND HORIZONTAL',
    '⛺': 'TENT',
    '🐕': 'DOG',
    '🏂': 'SNOWBOARDER',
    '👡': 'WOMANS SANDAL',
    '📻': 'RADIO',
    '✒': 'BLACK NIB',
    '🌰': 'CHESTNUT',
    '🏢': 'OFFICE BUILDING',
    '🎒': 'SCHOOL SATCHEL',
    '⌒': 'ARC',
    '🏫': 'SCHOOL',
    '📴': 'MOBILE PHONE OFF',
    '🚢': 'SHIP',
    '🚚': 'DELIVERY TRUCK',
    '🐉': 'DRAGON',
    '❒': 'UPPER RIGHT SHADOWED WHITE SQUARE',
    '🐊': 'CROCODILE',
    '🔔': 'BELL',
    '◢': 'BLACK LOWER RIGHT TRIANGLE',
    '🏥': 'HOSPITAL',
    '❔': 'WHITE QUESTION MARK ORNAMENT',
    '🚖': 'ONCOMING TAXI',
    '🃏': 'PLAYING CARD BLACK JOKER',
    '▼': 'BLACK DOWN-POINTING TRIANGLE',
    '▌': 'LEFT HALF BLOCK',
    '☛': 'BLACK RIGHT POINTING INDEX',
    '✩': 'STRESS OUTLINED WHITE STAR',
    '💒': 'WEDDING',
    '🚤': 'SPEEDBOAT',
    '🐐': 'GOAT',
    '■': 'BLACK SQUARE',
    '🔚': 'END WITH LEFTWARDS ARROW ABOVE',
    '🎻': 'VIOLIN',
    '🔷': 'LARGE BLUE DIAMOND',
    '🚦': 'VERTICAL TRAFFIC LIGHT',
    '🔓': 'OPEN LOCK',
    '🎽': 'RUNNING SHIRT WITH SASH',
    '📅': 'CALENDAR',
    '🎺': 'TRUMPET',
    '✯': 'PINWHEEL STAR',
    '🍈': 'MELON',
    '✉': 'ENVELOPE',
    '╣': 'BOX DRAWINGS DOUBLE VERTICAL AND LEFT',
    '◤': 'BLACK UPPER LEFT TRIANGLE',
    '○': 'WHITE CIRCLE',
    '🍼': 'BABY BOTTLE',
    '📀': 'DVD',
    '🚛': 'ARTICULATED LORRY',
    '📓': 'NOTEBOOK',
    '☉': 'SUN',
    '💴': 'BANKNOTE WITH YEN SIGN',
    '┼': 'BOX DRAWINGS LIGHT VERTICAL AND HORIZONTAL',
    '🐃': 'WATER BUFFALO',
    '➰': 'CURLY LOOP',
    '🔌': 'ELECTRIC PLUG',
    '🍄': 'MUSHROOM',
    '📕': 'CLOSED BOOK',
    '📣': 'CHEERING MEGAPHONE',
    '🚓': 'POLICE CAR',
    '🐗': 'BOAR',
    '↪': 'RIGHTWARDS ARROW WITH HOOK',
    '⛳': 'FLAG IN HOLE',
    '┻': 'BOX DRAWINGS HEAVY UP AND HORIZONTAL',
    '┛': 'BOX DRAWINGS HEAVY UP AND LEFT',
    '┃': 'BOX DRAWINGS HEAVY VERTICAL',
    '👱': 'PERSON WITH BLOND HAIR',
    '⏳': 'HOURGLASS WITH FLOWING SAND',
    '💺': 'SEAT',
    '🏇': 'HORSE RACING',
    '☻': 'BLACK SMILING FACE',
    '📞': 'TELEPHONE RECEIVER',
    'Ⓐ': 'CIRCLED LATIN CAPITAL LETTER A',
    '🌉': 'BRIDGE AT NIGHT',
    '🚩': 'TRIANGULAR FLAG ON POST',
    '✎': 'LOWER RIGHT PENCIL',
    '📃': 'PAGE WITH CURL',
    '🏨': 'HOTEL',
    '📌': 'PUSHPIN',
    '♎': 'LIBRA',
    '💷': 'BANKNOTE WITH POUND SIGN',
    '🚄': 'HIGH-SPEED TRAIN',
    '▲': 'BLACK UP-POINTING TRIANGLE',
    '⛵': 'SAILBOAT',
    '🔸': 'SMALL ORANGE DIAMOND',
    '⌛': 'HOURGLASS',
    '🚜': 'TRACTOR',
    '🐆': 'LEOPARD',
    '👒': 'WOMANS HAT',
    '❕': 'WHITE EXCLAMATION MARK ORNAMENT',
    '🔛': 'ON WITH EXCLAMATION MARK WITH LEFT RIGHT ARROW ABOVE',
    '♢': 'WHITE DIAMOND SUIT',
    '🇲': 'REGIONAL INDICATOR SYMBOL LETTER M',
    '❅': 'TIGHT TRIFOLIATE SNOWFLAKE',
    '👝': 'POUCH',
    '✞': 'SHADOWED WHITE LATIN CROSS',
    '◡': 'LOWER HALF CIRCLE',
    '🎋': 'TANABATA TREE',
    '👥': 'BUSTS IN SILHOUETTE',
    '📵': 'NO MOBILE PHONES',
    '🐡': 'BLOWFISH',
    '◆': 'BLACK DIAMOND',
    '🏯': 'JAPANESE CASTLE',
    '☂': 'UMBRELLA',
    '🔭': 'TELESCOPE',
    '🎪': 'CIRCUS TENT',
    '🐜': 'ANT',
    '♌': 'LEO',
    '☐': 'BALLOT BOX',
    '👷': 'CONSTRUCTION WORKER',
    '↳': 'DOWNWARDS ARROW WITH TIP RIGHTWARDS',
    '🔈': 'SPEAKER',
    '📄': 'PAGE FACING UP',
    '📍': 'ROUND PUSHPIN',
    '🚐': 'MINIBUS',
    '🚔': 'ONCOMING POLICE CAR',
    '🌋': 'VOLCANO',
    '📡': 'SATELLITE ANTENNA',
    '⏩': 'BLACK RIGHT-POINTING DOUBLE TRIANGLE',
    '🚳': 'NO BICYCLES',
    '✘': 'HEAVY BALLOT X',
    '۞': 'ARABIC START OF RUB EL HIZB',
    '☾': 'LAST QUARTER MOON',
    '🅰': 'NEGATIVE SQUARED LATIN CAPITAL LETTER A',
    '📥': 'INBOX TRAY',
    '🇼': 'REGIONAL INDICATOR SYMBOL LETTER W',
    '┓': 'BOX DRAWINGS HEAVY DOWN AND LEFT',
    '┣': 'BOX DRAWINGS HEAVY VERTICAL AND RIGHT',
    'Ⓛ': 'CIRCLED LATIN CAPITAL LETTER L',
    'Ⓔ': 'CIRCLED LATIN CAPITAL LETTER E',
    '🔦': 'ELECTRIC TORCH',
    '👤': 'BUST IN SILHOUETTE',
    '🚁': 'HELICOPTER',
    '🎠': 'CAROUSEL HORSE',
    '🐁': 'MOUSE',
    '📗': 'GREEN BOOK',
    '┐': 'BOX DRAWINGS LIGHT DOWN AND LEFT',
    '☮': 'PEACE SYMBOL',
    '♂': 'MALE SIGN',
    '◞': 'LOWER RIGHT QUADRANT CIRCULAR ARC',
    '📯': 'POSTAL HORN',
    '🔩': 'NUT AND BOLT',
    '👢': 'WOMANS BOOTS',
    '◂': 'BLACK LEFT-POINTING SMALL TRIANGLE',
    '📰': 'NEWSPAPER',
    '📶': 'ANTENNA WITH BARS',
    '🚥': 'HORIZONTAL TRAFFIC LIGHT',
    '🌄': 'SUNRISE OVER MOUNTAINS',
    '🗾': 'SILHOUETTE OF JAPAN',
    '🔶': 'LARGE ORANGE DIAMOND',
    '🏤': 'EUROPEAN POST OFFICE',
    '🎩': 'TOP HAT',
    'Ⓜ': 'CIRCLED LATIN CAPITAL LETTER M',
    '🔧': 'WRENCH',
    '🐅': 'TIGER',
    '♮': 'MUSIC NATURAL SIGN',
    '🅾': 'NEGATIVE SQUARED LATIN CAPITAL LETTER O',
    '🔄': 'ANTICLOCKWISE DOWNWARDS AND UPWARDS OPEN CIRCLE ARROWS',
    '☄': 'COMET',
    '☨': 'CROSS OF LORRAINE',
    '📦': 'PACKAGE',
    '🚊': 'TRAM',
    '🔲': 'BLACK SQUARE BUTTON',
    '🔁': 'CLOCKWISE RIGHTWARDS AND LEFTWARDS OPEN CIRCLE ARROWS',
    '△': 'WHITE UP-POINTING TRIANGLE',
    '📆': 'TEAR-OFF CALENDAR',
    '❛': 'HEAVY SINGLE TURNED COMMA QUOTATION MARK ORNAMENT',
    '📉': 'CHART WITH DOWNWARDS TREND',
    '▵': 'WHITE UP-POINTING SMALL TRIANGLE',
    '🔎': 'RIGHT-POINTING MAGNIFYING GLASS',
    '☜': 'WHITE LEFT POINTING INDEX',
    '🇯': 'REGIONAL INDICATOR SYMBOL LETTER J',
    '🇵': 'REGIONAL INDICATOR SYMBOL LETTER P',
    '📘': 'BLUE BOOK',
    '✡': 'STAR OF DAVID',
    'ⓔ': 'CIRCLED LATIN SMALL LETTER E',
    '🔑': 'KEY',
    '🔃': 'CLOCKWISE DOWNWARDS AND UPWARDS OPEN CIRCLE ARROWS',
    '👃': 'NOSE',
    '⭕': 'HEAVY LARGE CIRCLE',
    '🔘': 'RADIO BUTTON',
    'ⓒ': 'CIRCLED LATIN SMALL LETTER C',
    '🚭': 'NO SMOKING SYMBOL',
    '🚉': 'STATION',
    '🚪': 'DOOR',
    '➳': 'WHITE-FEATHERED RIGHTWARDS ARROW',
    '🚃': 'RAILWAY CAR',
    '┯': 'BOX DRAWINGS DOWN LIGHT AND HORIZONTAL HEAVY',
    '🏬': 'DEPARTMENT STORE',
    '☽': 'FIRST QUARTER MOON',
    '🆙': 'SQUARED UP WITH EXCLAMATION MARK',
    '🆖': 'SQUARED NG',
    '☪': 'STAR AND CRESCENT',
    '┗': 'BOX DRAWINGS HEAVY UP AND RIGHT',
    '🚮': 'PUT LITTER IN ITS PLACE SYMBOL',
    '┫': 'BOX DRAWINGS HEAVY VERTICAL AND LEFT',
    'Ⓞ': 'CIRCLED LATIN CAPITAL LETTER O',
    '❇': 'SPARKLE',
    '✴': 'EIGHT POINTED BLACK STAR',
    '┌': 'BOX DRAWINGS LIGHT DOWN AND RIGHT',
    '☊': 'ASCENDING NODE',
    '🔕': 'BELL WITH CANCELLATION STROKE',
    '⬛': 'BLACK LARGE SQUARE',
    '❝': 'HEAVY DOUBLE TURNED COMMA QUOTATION MARK ORNAMENT',
    '❞': 'HEAVY DOUBLE COMMA QUOTATION MARK ORNAMENT',
    '🚞': 'MOUNTAIN RAILWAY',
    '🍶': 'SAKE BOTTLE AND CUP',
    '🌐': 'GLOBE WITH MERIDIANS',
    '♀': 'FEMALE SIGN',
    '🚅': 'HIGH-SPEED TRAIN WITH BULLET NOSE',
    '🚒': 'FIRE ENGINE',
    '➣': 'THREE-D BOTTOM-LIGHTED RIGHTWARDS ARROWHEAD',
    '♋': 'CANCER',
    '♍': 'VIRGO',
    '🕝': 'CLOCK FACE TWO-THIRTY',
    'ⓐ': 'CIRCLED LATIN SMALL LETTER A',
    '✗': 'BALLOT X',
    '📙': 'ORANGE BOOK',
    'Ⓢ': 'CIRCLED LATIN CAPITAL LETTER S',
    '📋': 'CLIPBOARD',
    '⇢': 'RIGHTWARDS DASHED ARROW',
    '🎱': 'BILLIARDS',
    '🐞': 'LADY BEETLE',
    '🔺': 'UP-POINTING RED TRIANGLE',
    'ⓡ': 'CIRCLED LATIN SMALL LETTER R',
    '🎍': 'PINE DECORATION',
    '♤': 'WHITE SPADE SUIT',
    '🎲': 'GAME DIE',
    '🎯': 'DIRECT HIT',
    '〠': 'POSTAL MARK FACE',
    '🔉': 'SPEAKER WITH ONE SOUND WAVE',
    '↩': 'LEFTWARDS ARROW WITH HOOK',
    '🚾': 'WATER CLOSET',
    '🎣': 'FISHING POLE AND FISH',
    '🔣': 'INPUT SYMBOL FOR SYMBOLS',
    '❎': 'NEGATIVE SQUARED CROSS MARK',
    '➥': 'HEAVY BLACK CURVED DOWNWARDS AND RIGHTWARDS ARROW',
    '🅱': 'NEGATIVE SQUARED LATIN CAPITAL LETTER B',
    '🎌': 'CROSSED FLAGS',
    '◣': 'BLACK LOWER LEFT TRIANGLE',
    '⏬': 'BLACK DOWN-POINTING DOUBLE TRIANGLE',
    '♭': 'MUSIC FLAT SIGN',
    '💠': 'DIAMOND SHAPE WITH A DOT INSIDE',
    'ⓞ': 'CIRCLED LATIN SMALL LETTER O',
    '🔳': 'WHITE SQUARE BUTTON',
    '🏭': 'FACTORY',
    '🔰': 'JAPANESE SYMBOL FOR BEGINNER',
    '🎳': 'BOWLING',
    '☚': 'BLACK LEFT POINTING INDEX',
    '➽': 'HEAVY WEDGE-TAILED RIGHTWARDS ARROW',
    '➫': 'BACK-TILTED SHADOWED WHITE RIGHTWARDS ARROW',
    '➖': 'HEAVY MINUS SIGN',
    '🏮': 'IZAKAYA LANTERN',
    '📛': 'NAME BADGE',
    '꒰': 'YI RADICAL SHY',
    '꒱': 'YI RADICAL VEP',
    '◝': 'UPPER RIGHT QUADRANT CIRCULAR ARC',
    '📑': 'BOOKMARK TABS',
    '🎦': 'CINEMA',
    'ⓧ': 'CIRCLED LATIN SMALL LETTER X',
    '🇨': 'REGIONAL INDICATOR SYMBOL LETTER C',
    '🇳': 'REGIONAL INDICATOR SYMBOL LETTER N',
    '🔟': 'KEYCAP TEN',
    '〓': 'GETA MARK',
    'ⓜ': 'CIRCLED LATIN SMALL LETTER M',
    '➠': 'HEAVY DASHED TRIANGLE-HEADED RIGHTWARDS ARROW',
    '🚆': 'TRAIN',
    '🚠': 'MOUNTAIN CABLEWAY',
    '℅': 'CARE OF',
    '☃': 'SNOWMAN',
    '🚽': 'TOILET',
    '📐': 'TRIANGULAR RULER',
    'ⓝ': 'CIRCLED LATIN SMALL LETTER N',
    '✮': 'HEAVY OUTLINED BLACK STAR',
    '⇦': 'LEFTWARDS WHITE ARROW',
    '👲': 'MAN WITH GUA PI MAO',
    '🚡': 'AERIAL TRAMWAY',
    '🎑': 'MOON VIEWING CEREMONY',
    '🔬': 'MICROSCOPE',
    '➗': 'HEAVY DIVISION SIGN',
    '📈': 'CHART WITH UPWARDS TREND',
    '⌘': 'PLACE OF INTEREST SIGN',
    '⏪': 'BLACK LEFT-POINTING DOUBLE TRIANGLE',
    '╹': 'BOX DRAWINGS HEAVY UP',
    '◎': 'BULLSEYE',
    '🔼': 'UP-POINTING SMALL RED TRIANGLE',
    '꒦': 'YI RADICAL GGUO',
    '📎': 'PAPERCLIP',
    '⑅': 'OCR BOW TIE',
    '⍝': 'APL FUNCTIONAL SYMBOL UP SHOE JOT',
    '📁': 'FILE FOLDER',
    '✭': 'OUTLINED BLACK STAR',
    '➲': 'CIRCLED HEAVY WHITE RIGHTWARDS ARROW',
    '♓': 'PISCES',
    '┏': 'BOX DRAWINGS HEAVY DOWN AND RIGHT',
    '☇': 'LIGHTNING',
    '♺': 'RECYCLING SYMBOL FOR GENERIC MATERIALS',
    '♞': 'BLACK CHESS KNIGHT',
    '࿎': 'TIBETAN SIGN RDEL NAG RDEL DKAR',
    '📠': 'FAX MACHINE',
    '👘': 'KIMONO',
    '↙': 'SOUTH WEST ARROW',
    'Ⓕ': 'CIRCLED LATIN CAPITAL LETTER F',
    'Ⓦ': 'CIRCLED LATIN CAPITAL LETTER W',
    'Ⓟ': 'CIRCLED LATIN CAPITAL LETTER P',
    '🕑': 'CLOCK FACE TWO OCLOCK',
    '💽': 'MINIDISC',
    '🕛': 'CLOCK FACE TWELVE OCLOCK',
    '🎫': 'TICKET',
    '♈': 'ARIES',
    '📟': 'PAGER',
    '℃': 'DEGREE CELSIUS',
    '↬': 'RIGHTWARDS ARROW WITH LOOP',
    '🕒': 'CLOCK FACE THREE OCLOCK',
    '🇰': 'REGIONAL INDICATOR SYMBOL LETTER K',
    '↱': 'UPWARDS ARROW WITH TIP RIGHTWARDS',
    '✍': 'WRITING HAND',
    '⇐': 'LEFTWARDS DOUBLE ARROW',
    '🏦': 'BANK',
    '🔻': 'DOWN-POINTING RED TRIANGLE',
    'ⓟ': 'CIRCLED LATIN SMALL LETTER P',
    'ⓕ': 'CIRCLED LATIN SMALL LETTER F',
    'ⓘ': 'CIRCLED LATIN SMALL LETTER I',
    '♿': 'WHEELCHAIR SYMBOL',
    '⇗': 'NORTH EAST DOUBLE ARROW',
    '⇘': 'SOUTH EAST DOUBLE ARROW',
    'ⓨ': 'CIRCLED LATIN SMALL LETTER Y',
    'ⓙ': 'CIRCLED LATIN SMALL LETTER J',
    '▫': 'WHITE SMALL SQUARE',
    '🔇': 'SPEAKER WITH CANCELLATION STROKE',
    '⌃': 'UP ARROWHEAD',
    '🔖': 'BOOKMARK',
    '📜': 'SCROLL',
    '♏': 'SCORPIUS',
    '🚝': 'MONORAIL',
    '☢': 'RADIOACTIVE SIGN',
    '🎏': 'CARP STREAMER',
    '┘': 'BOX DRAWINGS LIGHT UP AND LEFT',
    '✝': 'LATIN CROSS',
    '❖': 'BLACK DIAMOND MINUS WHITE X',
    '⍣': 'APL FUNCTIONAL SYMBOL STAR DIAERESIS',
    '📮': 'POSTBOX',
    '🕕': 'CLOCK FACE SIX OCLOCK',
    '🇭': 'REGIONAL INDICATOR SYMBOL LETTER H',
    '◜': 'UPPER LEFT QUADRANT CIRCULAR ARC',
    '🔯': 'SIX POINTED STAR WITH MIDDLE DOT',
    '➸': 'HEAVY BLACK-FEATHERED RIGHTWARDS ARROW',
    '꒵': 'YI RADICAL JJY',
    '🕥': 'CLOCK FACE TEN-THIRTY',
    '♙': 'WHITE CHESS PAWN',
    '▿': 'WHITE DOWN-POINTING SMALL TRIANGLE',
    '⚃': 'DIE FACE-4',
    '✽': 'HEAVY TEARDROP-SPOKED ASTERISK',
    '📼': 'VIDEOCASSETTE',
    '🕐': 'CLOCK FACE ONE OCLOCK',
    '🀄': 'MAHJONG TILE RED DRAGON',
    '✾': 'SIX PETALLED BLACK AND WHITE FLORETTE',
    '✬': 'BLACK CENTRE WHITE STAR',
    '🆑': 'SQUARED CL',
    '✫': 'OPEN CENTRE BLACK STAR',
    '🕔': 'CLOCK FACE FIVE OCLOCK',
    '❣': 'HEAVY HEART EXCLAMATION MARK ORNAMENT',
    '➱': 'NOTCHED UPPER RIGHT-SHADOWED WHITE RIGHTWARDS ARROW',
    '🆕': 'SQUARED NEW',
    '➢': 'THREE-D TOP-LIGHTED RIGHTWARDS ARROWHEAD',
    '↕': 'UP DOWN ARROW',
    '📫': 'CLOSED MAILBOX WITH RAISED FLAG',
    '🉐': 'CIRCLED IDEOGRAPH ADVANTAGE',
    '♊': 'GEMINI',
    '🈂': 'SQUARED KATAKANA SA',
    '🎰': 'SLOT MACHINE',
    '҂': 'CYRILLIC THOUSANDS SIGN',
    '╤': 'BOX DRAWINGS DOWN SINGLE AND HORIZONTAL DOUBLE',
    '➛': 'DRAFTING POINT RIGHTWARDS ARROW',
    '♝': 'BLACK CHESS BISHOP',
    '❋': 'HEAVY EIGHT TEARDROP-SPOKED PROPELLER ASTERISK',
    '✆': 'TELEPHONE LOCATION SIGN',
    '📔': 'NOTEBOOK WITH DECORATIVE COVER'
}


In [7]:
import emoji_data_python

list_emoji_name = list(emoji2unicode_name.values())
dico_emoji = {}
for x in list_emoji_name:
    if emoji_data_python.find_by_name(x) != []:
        dico_emoji[str(emoji_data_python.find_by_name(x)[0].text)] = x
dico_emoji['(:'] = "HAPPY FACE"
dico_emoji['<3'] = "HEART"
print(dico_emoji)

{'None': 'NOTEBOOK WITH DECORATIVE COVER', '<3': 'HEART', ":'(": 'CRYING FACE', ':)': 'FACE WITH OPEN MOUTH', ';)': 'WINKING FACE', ':(': 'DISAPPOINTED FACE', ';p': 'FACE WITH STUCK-OUT TONGUE AND WINKING EYE', ':D': 'GRINNING FACE', '</3': 'BROKEN HEART', ':p': 'FACE WITH STUCK-OUT TONGUE', '(:': 'HAPPY FACE'}


In [8]:
def deal_with_emojis(content: List) :
    list_of_sentences = []
    for sentence in content :
        text = list(sentence)[0]
        bow = text.split(" ")
        for i in range(len(bow)) :
            for emoji in emoji2unicode_name :
                if emoji in bow[i]:
                    bow[i] = bow[i].replace(emoji,emoji2unicode_name[emoji])
            for emoji in dico_emoji :
                if emoji in bow[i]:
                    bow[i] = bow[i].replace(emoji,dico_emoji[emoji])
        text_processed = ""
        for i in range(len(bow)):
            text_processed = text_processed + bow[i] + " "
        list_of_sentences.append(text_processed)
    return(list_of_sentences)

final = deal_with_emojis(content_list)



In [9]:
f = open("data_processed_after.txt","w+")
for sentence in final:
    f.write(sentence + '\n')


In [10]:
def split_QA(content: list):
    QA = []
    other = []
    for x in content:
        if "@" in x:
            QA.append(x)
        else:
            other.append(x)
    return(QA, other)

split_QA(final)

(['Trop bien un live chill merci @Sardoche ',
  'Merci pour LuvOops @mrstro ',
  '@Sardoche Salut mon Sardo VoHiYo ',
  "@sarukog sardWeeb bg;t'as win ton bo ? ",
  "@CrIKxS oep ez; 17 euw Vel'Koz en plus Keepo ",
  '@sarukog pololo trop chaud wp GRINNING FACE ',
  "T'es à combien de tour de bras @sardoche ? ",
  'Si y a une musique on ne l’entend pas si jamais @sardoche ',
  'Merci pour LuvBlondeR @nooxford ',
  'enfin tu streams; tu tes crus au 35h ou quoi ? @Sardoche ',
  '@Sardoche quel est le secret de tes gros bras ? ',
  '@leboeufbourguignons tu donnes faim ',
  '@lepetitzebre sardHi ',
  '@Sardoche yo ducoup demain soir tu stream stp ? ',
  "alors la run d'hier ? @sardoche ",
  "@Sardoche Spn d'alerte très bas ! ",
  '@sarukog Salut VoHiYo ',
  '@h3ylloo sardWeeb ',
  'et là le RNG va te niquer ta journée LUL @Sardoche ',
  'Le focus @Sardoche ',
  'Que ta voix @Sardoche ',
  'oui @sardoche bonsoir ',
  'yo @Sardoche GL HF ',
  "@Flyyyy__ t'as un probleme ? ",
  '@Yukasoro ok '